In [19]:
import pandas as pd
import sim
#AI was utilized to devolop the simulation and plots

**Assumptions:**

- Risk-free rate: $r_f = 0.04$
- Bitcoin mining rate per rig: $\text{Mine rate} = 0.00008$ BTC/day
- Power consumption per rig: $3{,}250$ W/day
- Number of Antminers: $1{,}000$
- Market utilized: Day Ahead
- M_Paths = 500

<h2>
Node: NSPNEWLOAD

In [20]:
nsp= sim.run_sim(seed=None, intraday=False, analysis = False, plot_vol_surface=False, Node='NSP')

LSMC optimal NPV : $891,565


<h2> Intraday Analysis

**Assumptions:**

- Intraday Period = 15M

In [21]:
intra = sim.run_sim(seed=None, intraday=True, analysis = True, plot_vol_surface=True, Node='NSP')

LSMC optimal NPV : $558,800


In [22]:
#Print Summary

print(f"Always operating ${intra['Operate all ($)'].values[0]}")
print(f"Least Squares Monte Carlo Payout ${intra['Net payout ($)'].values[0]} ")
print(f"Curtail Value ${intra['Curtail value ($)'].values[0]}")

Always operating $521962
Least Squares Monte Carlo Payout $558800 
Curtail Value $36838


This 3D surface plot illustrates how the net payout from a BTC mining operation varies with changes in BTC price volatility  and electricity price volatility. Generally, we observe that higher BTC volatility tends to increase the expected payout. This is unexpected as we would think greater volatility in BTC would decrease the net payouts or when it is optimal to exercise. In contrast, increased electricity volatility introduces more uncertainty in operating costs, often leading to lower or more erratic payouts due to the risk of unexpected price spikes. However the surface shows several sharp peaks, indicating that certain combinations of volatilities can lead to significantly higher payouts, though these can be driven by random shocks in the model. Although the randomization and shocks may oppose the explaination

<H2>
New Node: ODEL

In [23]:
odel = sim.run_sim(seed=None, intraday=False, analysis = False, plot_vol_surface=False, Node='ODEL')

LSMC optimal NPV : $763,999


In [24]:
#Average Annual Profit
profit = odel['npv_lsm']

#Volatility in profit
vol = odel['npv_lst_std']

#Sharpe
sharpe = profit / vol

print(f"Average Profit: ${profit:.2f}")
print(f"Volatility: {vol:.0f}")
print(f"Sharpe Ratio: {sharpe:.2f}")

Average Profit: $763998.97
Volatility: 579707
Sharpe Ratio: 1.32


# Final Project: Evaluating a Crypto Mining Data Center

## Part 1. The first facility

**Across >500 simulations, what is the expected average annual excess profit?**


In [25]:
profit = nsp['npv_lsm']
print(f"${nsp['npv_lsm']:.2f}")

$891564.68


**Across the simulations what is the volatility in annual profitability?**

In [26]:
vol = nsp['npv_lst_std']
print(f"{nsp['npv_lst_std']:.2f}")

925109.65


**What is the ratio of this average excess profitability to volatility?**

In [27]:
print(profit / vol)

0.9637394633770381


**Antminer’s S19 pros are selling for ~$500 USD$.
If your uncle offered to sell you the facility for $500K$, would you buy it?
Based on the return given the respective volatility of BTC versus electricity, what is the breakeven expected value of the facility?**

Assuming a risk-free rate of 4% and expected average annual profits of $908,810.69$, the choice is trivial: we would buy the facility for $500,000$.

However, if we factor in volatility, we observe that the profits are widely dispersed across simulations. As a result, there's a meaningful chance of incurring a steep loss even if the average return is high. Assuming a normal distribution of returns across 500 simulations, the present value (discounted at 4%) of the annual profit falls below $500,000 in approximately 160 simulations.

Considering this tail risk, we would still proceed with the purchase, although the decision now depends on one’s level of risk aversion.

**What is the simulated average annual excess profit expectation?**

In [28]:
profit_odel = odel['npv_lsm']
print(f"Average Profit: ${profit_odel:.2f}")

Average Profit: $763998.97


**What is the simulated excess profit-to-volatility ratio?**

In [29]:
#Volatility in profit
vol_odel = odel['npv_lst_std']

#Sharpe
sharpe = profit_odel / vol_odel

print(sharpe)

1.317906071383213


**How does this investment compare to doubling the size of the installation at your uncle’s original location (assuming the same capital expenditure in each case)? What can you learn from this difference?**
$$NPVdoubled = 2 * NPVoriginal$$
$$Sharpedoubled = Sharpeoriginal$$
This is probably a discussion of fixed costs vs marginal costs.

### 3 Understanding the Model

We arrive at an answer using the cost minimization function:

$$
\min_K \left[ K + qC(K) \right],
$$

where \( K \) is cost of capital investment, \( q \) is the fraction of the year we are operating, and \( C \) is cost of operating per KWH. For example, if we run the plant for the whole year \( i \), then we have costs:

$$
K_i + C_i.
$$

Intuitively, we assume that \( C(K) \) is a decreasing cost function such that its derivative is \( C'(K) < 0 \).

Solving the minimization problem, we reach the following result:

$$
\frac{\partial}{\partial K} \left( K + qC(K) \right) = 0
\quad \Longrightarrow \quad
1 + qC'(K) = 0 
\quad \Longrightarrow \quad
C'(K) = -\frac{1}{q}
$$

If we assume \( q = 1 \) (whole year), then the optimal level of \( K \), \( K^* \), is where \( C'(K) = -1 \). In the graph below, the tradeoff mentioned in the intro becomes visible.